Homework 2 Part 2 (due 7/07/2024)
Authors: Sri Korandla, Warren Shepard, Tushar Agarwal
Credit: pgmpy documentation: https://pgmpy.org
Date: 05 July 2024

# Health-care assistance via probabilistic graphical modeling

### Objective
In this project, you will create a health-care assistance bot that can suggest diagnoses for a set of symptoms based on a probabilistic graphical model.

### Step 1: Review
Review the code from the Bayesian networks exercise.

### Step 2: Acquire data
View this [research article](https://www.nature.com/articles/ncomms5212) and download its supplementary data sets 1, 2 and 3. These data sets include the occurrences of diseases, symptoms, and their co-occurrences in the scientific literature. (For the purpose of this exercise, we are going to assume that the frequency of co-occurrences of diseases and symptoms in scientific papers is proportional to the co-occurence frequencies of actual disease cases and symptoms.)

### Step 3: Create a Bayesian network
Using commands from the `pgmpy` library, create a Bayesian network in which the probability of exhibiting a symptom is conditional on the probability of having an associated disease.






In [1]:
# imports
!pip install pgmpy
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

IMPORTAND: it is assumed that the files are called `diseases.txt`, `symptoms.txt`, and `cooccurences.txt`

In [2]:
# STEP 2
# Load in Data, separate by tab between symptom, disease, and occurences, and omit first line
diseases_df = pd.read_csv('diseases.txt', sep='\t', header=None, names=['Disease', 'Occurrences'], skiprows=1)
symptoms_df = pd.read_csv('symptoms.txt', sep='\t', header=None, names=['Symptom', 'Occurrences'], skiprows=1, on_bad_lines='skip')
cooccurrences_df = pd.read_csv('cooccurences.txt', sep='\t', header=None, names=['Symptom', 'Disease', 'Number', 'TFIDF'], skiprows=1, on_bad_lines='skip')

# Filter cooccurrences with at least 500 occurrences (did 500 becuase 300 took tool)
cooccurrences_df = cooccurrences_df[cooccurrences_df['Number'] >= 500]

# Calculate the sum of occurrences for diseases and symptoms
disease_occurrences = cooccurrences_df.groupby('Disease')['Number'].sum().reset_index()
symptom_occurrences = cooccurrences_df.groupby('Symptom')['Number'].sum().reset_index()

# Update the occurrences in the diseases_df
disease_occurrences_dict = dict(zip(disease_occurrences['Disease'], disease_occurrences['Number']))
diseases_df['Occurrences'] = diseases_df['Disease'].map(disease_occurrences_dict).fillna(0).astype(int)

# Update the occurrences in the symptoms_df
symptom_occurrences_dict = dict(zip(symptom_occurrences['Symptom'], symptom_occurrences['Number']))
symptoms_df['Occurrences'] = symptoms_df['Symptom'].map(symptom_occurrences_dict).fillna(0).astype(int)

# Remove rows with no occurrences
diseases_df = diseases_df[diseases_df['Occurrences'] > 0]
symptoms_df = symptoms_df[symptoms_df['Occurrences'] > 0]

# symptoms_in_cooccurrences = cooccurrences_df['Symptom'].unique()
# symptoms_in_symptoms_df = symptoms_df['Symptom'].unique()
# symptoms_not_in_symptoms_df = set(symptoms_in_cooccurrences) - set(symptoms_in_symptoms_df)


print(diseases_df)
print(symptoms_df)
print(cooccurrences_df)


                      Disease  Occurrences
1                Hypertension         8758
2     Coronary Artery Disease         8603
4       Myocardial Infarction         6216
6            Coronary Disease         7066
7                      Asthma         2448
...                       ...          ...
2286         Fetal Macrosomia          581
2302                Athetosis          569
2345              Presbycusis          543
2382              Tinea Pedis          518
2400       Dyslexia, Acquired          509

[243 rows x 2 columns]
                           Symptom  Occurrences
0                      Body Weight        21119
1                             Pain        57615
2                          Obesity        68836
3                           Anoxia         3115
4               Mental Retardation        22278
..                             ...          ...
199                    Presbycusis          543
200                       Myotonia          605
208                  Gastrop

In [3]:
# STEP 3
# create ModeL, create list of diseases + symptoms, and add edge if coocurrence
model = BayesianNetwork()

# add lables for diseases vs somptoms becuase there are many duplicates
diseases_df['Disease'] = 'D_' + diseases_df['Disease']
symptoms_df['Symptom'] = 'S_' + symptoms_df['Symptom']
cooccurrences_df['Disease'] = 'D_' + cooccurrences_df['Disease']
cooccurrences_df['Symptom'] = 'S_' + cooccurrences_df['Symptom']

diseases = diseases_df['Disease'].tolist()
symptoms = symptoms_df['Symptom'].tolist()

# adding edges between co-occuring diseases and symptoms
for index, row in cooccurrences_df.iterrows():
  disease = row['Disease']
  symptom = row['Symptom']

  # add disease and symptom to network if needed
  if not model.has_node(disease):
    model.add_node(disease)
  if not model.has_node(symptom):
    model.add_node(symptom)

  model.add_edge(disease, symptom)

print(model.edges())
print(model.nodes())


[('D_Bacterial Infections', 'S_Fever'), ('D_Neutropenia', 'S_Fever'), ('D_Kidney Failure, Chronic', 'S_Body Weight'), ('D_Kidney Failure, Chronic', 'S_Proteinuria'), ('D_Hypertension', 'S_Body Weight'), ('D_Hypertension', 'S_Obesity'), ('D_Hypertension', 'S_Angina Pectoris'), ('D_Hypertension', 'S_Proteinuria'), ('D_Hypertension', 'S_Albuminuria'), ('D_Diabetes Mellitus', 'S_Body Weight'), ('D_Diabetes Mellitus', 'S_Obesity'), ('D_Diabetes Mellitus, Experimental', 'S_Body Weight'), ('D_Diabetes Mellitus, Type 1', 'S_Body Weight'), ('D_Diabetes Mellitus, Type 1', 'S_Albuminuria'), ('D_Diabetes Mellitus, Type 2', 'S_Body Weight'), ('D_Diabetes Mellitus, Type 2', 'S_Weight Loss'), ('D_Diabetes Mellitus, Type 2', 'S_Obesity'), ('D_Diabetes Mellitus, Type 2', 'S_Albuminuria'), ('D_Nutrition Disorders', 'S_Body Weight'), ('D_Protein-Energy Malnutrition', 'S_Body Weight'), ('D_Obesity', 'S_Body Weight'), ('D_Obesity', 'S_Weight Gain'), ('D_Obesity', 'S_Weight Loss'), ('D_Obesity', 'S_Overweig

### Step 4: Initialize priors
Use the disease occurrence data to assign prior probabilities for diseases.

In [5]:
# STEP 4
# Initialize priors
total_occurrences = diseases_df['Occurrences'].sum()

for disease in diseases:
    disease_occurrences = diseases_df[diseases_df['Disease'] == disease]['Occurrences'].values[0]
    prob_disease = disease_occurrences / total_occurrences
    cpd_disease = TabularCPD(variable=disease, variable_card=2, values=[[1 - prob_disease], [prob_disease]])
    model.add_cpds(cpd_disease)

print(total_occurrences)

684145


### Step 5: Calculate conditional probability tables
Use the co-occurrence data to define CPTs for each connected pair of disease and symptoms. (Hint: You may need to assign some occurrences of symptoms to an "idiopathic disease" to create valid CPTs.)

In [6]:
def grabFromDF(df, value, iv_index=0, dv_index=1):
  '''
  From a 2-column data frame, look for the row in which the value is of
  the first column is `value`. Return the corresponding value of the second
  column.
  '''
  sub_df = (df[df[df.columns[iv_index]] == value][df.columns[dv_index]])
  if len(sub_df):
    return sub_df.iloc[0]
  else:
    return 0

def grabFromDF2(df, value1, value2, iv1_index=0, iv2_index=1, dv_index=2):
  '''
  From a 3-column data frame, look for the row in which the value is of
  the first column is `value1` AND the value of the second column in
  `value2`. Return the corresponding value of the third column.
  '''
  # select variables
  var1 = df.columns[iv1_index]
  var2 = df.columns[iv2_index]
  var3 = df.columns[dv_index]

  # select sub-dataframes in which these variables have the desired values
  sub_df = df[df[var1] == value1]
  subsub_df = sub_df[sub_df[var2] == value2]

  # return dependent variable
  if len(subsub_df):
    return subsub_df[var3].iloc[0]
  else:
    return 0

def CPT2x2(disease_occ, symptom_occ, interaction_occ,
           total_disease_occ, total_symptom_occ, total_interaction_occ):
  '''
  Set the 2x2 CPTs for a disease-symptom pair based on the occurrences
  of the disease, symptom, their cooccurrences and the total occurrences
  of disease, symptoms, and interactions in the data set.
  '''

  # probability of disease
  p_disease = disease_occ / total_disease_occ
  # joint probability of symptom and disease occurring
  p_joint = interaction_occ / total_interaction_occ

  # conditional probability of symptom occurrence given disease
  pTT = (p_joint / p_disease if p_joint > 0 else 0.0)
  # conditional probability of symptom non-occurrence given disease
  pFT = 1 - pTT
  # conditional probability of symptom occurrence given disease absence
  pTF = (symptom_occ - interaction_occ) / total_symptom_occ
  # conditional probability of symptom non-occurrence given disease absence
  pFF = 1 - pTF

  return [pFF, pTF, pFT, pTT]


In [7]:
import itertools
import numpy as np

In [8]:
all_symptoms = symptoms_df['Symptom'].unique().tolist()

# Define CPTs for symptom nodes
CPTs_symptoms = []

total_disease_occurrences = diseases_df['Occurrences'].sum()
total_symptom_occurrences = symptoms_df['Occurrences'].sum()
total_interaction_occurrences = cooccurrences_df['Number'].sum()

for i0, symptom in enumerate(all_symptoms):
    # get all parent nodes
    parents = list(model.predecessors(symptom))
    print(len(parents), end=' ')

    # collect 2x2 CPTs for each parent
    little_cpts = []

    for disease in parents:
        # occurrence of the selected disease
        disease_occurrence = grabFromDF(diseases_df, disease)
        # occurrence of the selected symptom
        symptom_occurrence = grabFromDF(symptoms_df, symptom)
        # occurrence of interaction
        interaction_occurrence = grabFromDF2(cooccurrences_df, symptom, disease)

        little_cpt = CPT2x2(disease_occurrence, symptom_occurrence,
                            interaction_occurrence, total_disease_occurrences,
                            total_symptom_occurrences, total_interaction_occurrences)
        # add 2x2-CPT to list of 2x2-CPTs
        little_cpts += [little_cpt]

    # For the purpose of this exercise, we are going to assume that the
    # occurrence of one disease is always independent of the occurrence of
    # another disease (i.e., no comorbidities, naive Bayes).

    rowT = [] # row of probabilities where symptom == True
    for bool_combo in itertools.product([0,1], repeat=len(parents)):
        cond_probs = [little_cpts[i][2+b] for i, b in enumerate(bool_combo)]
        rowT += [np.prod(cond_probs)]

    rowF = [1-val for val in rowT] # row of probs where symptom == False

    cpt = TabularCPD(variable=symptom, variable_card=2, values=[rowF, rowT],
                     evidence=parents, evidence_card=[2 for _ in parents])
    CPTs_symptoms += [cpt]
    model.add_cpds(cpt)


12 12 12 4 6 3 5 7 1 4 2 2 1 2 4 3 1 8 1 1 1 1 2 4 1 1 5 3 1 1 2 2 3 7 2 3 1 1 1 1 4 2 3 2 1 1 4 2 6 1 2 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 1 2 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 4 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [9]:
model.check_model()

True

In [ ]:
# IMPORTANT: the below is just some of my testing mechanisms--can ignore!
disease_to_symptoms = cooccurrences_df.groupby('Disease')['Symptom'].apply(list).to_dict()
print(disease_to_symptoms)

disease_with_most_symptoms = max(disease_to_symptoms, key=lambda k: len(disease_to_symptoms[k]))
print(disease_with_most_symptoms)
print(disease_to_symptoms[disease_with_most_symptoms])

{'D_Abnormalities, Multiple': ['S_Mental Retardation'], 'D_Acute Coronary Syndrome': ['S_Acute Coronary Syndrome'], 'D_Agnosia': ['S_Agnosia'], 'D_Albuminuria': ['S_Albuminuria'], 'D_Altitude Sickness': ['S_Anoxia'], 'D_Alzheimer Disease': ['S_Memory Disorders'], 'D_Amblyopia': ['S_Amblyopia'], 'D_Amnesia': ['S_Amnesia'], 'D_Anemia, Sickle Cell': ['S_Pain'], 'D_Angina Pectoris': ['S_Angina Pectoris', 'S_Angina, Unstable'], 'D_Angina Pectoris, Variant': ['S_Angina Pectoris, Variant'], 'D_Angina, Unstable': ['S_Angina Pectoris', 'S_Angina, Unstable'], 'D_Anorexia Nervosa': ['S_Body Weight', 'S_Bulimia'], 'D_Aphasia': ['S_Aphasia', 'S_Aphasia, Broca'], 'D_Aphasia, Broca': ['S_Aphasia, Broca'], 'D_Apnea': ['S_Apnea'], 'D_Apraxias': ['S_Apraxias'], 'D_Arrhythmias, Cardiac': ['S_Syncope'], 'D_Arterial Occlusive Diseases': ['S_Intermittent Claudication'], 'D_Arthralgia': ['S_Arthralgia'], 'D_Arthritis, Rheumatoid': ['S_Pain'], 'D_Articulation Disorders': ['S_Articulation Disorders'], 'D_Asthm

In [ ]:
# IMPORTNAT: also some testing mechanisms--can ignore!
inference = VariableElimination(model)

# 1 means has symptom
# 0 means does not have symptom
evidence = {'S_Weight Gain': 1,'S_Overweight': 1, 'S_Obesity': 1}  # Symptoms observed as present
query_result = inference.query(variables=['D_Obesity'], evidence=evidence)
print(query_result)

+--------------+------------------+
| D_Obesity    |   phi(D_Obesity) |
+==============+==================+
| D_Obesity(0) |           0.9998 |
+--------------+------------------+
| D_Obesity(1) |           0.0002 |
+--------------+------------------+


### Step 6:
Create a minimal interface in which your bot asks a users for a list of observed symptoms and then returns the name of the disease that is the most likely match to the symptoms. (Hint: Review the input/output commands that you have used in last week's homework.)

In [10]:
# Time to Perform Inferences as a test!

inference = VariableElimination(model)
print("symptoms start with an S_ before the symptom. Ex: S_Body Weight")
symptom_input = input("enter symptoms, separated by commas: ")
symptom_list = [symptom.strip() for symptom in symptom_input.split(',')] # put user input into list
# symptom_list = ['S_Body Weight'] # sample input for symptom_list can be S_Body Weight


def checkSymptoms():
  # check if symptoms not in list
  model_variables = model.nodes()
  missing_symptoms = [symptom for symptom in symptom_list if symptom not in model_variables] # is symptoms are not in list
  if missing_symptoms:
    print('\nSome of your symptoms were not in the database. Check your spelling.')
    print('The following were not present:')
    for symptom in missing_symptoms:
      print(symptom)
    return False
  return True
passed = checkSymptoms()

def findMostLikely():
  highest_prob = 0
  most_likley_disease = None
  diseases = diseases_df['Disease'].tolist()
  # diseases = 'D_Hypertension' exaample
  for disease in diseases:
    result = inference.query(variables= symptom_list, evidence={disease: 1})
     # Access the last column and last row of the result table
    last_col_index = -1  # Assuming you want the last column
    last_row_index = -1  # Assuming you want the last row
    last_col = result.values[..., last_col_index]
    last_row = result.values[last_row_index, ...]
    # Get the highest probability from the last column and last row
    max_prob_last_col = last_col.max()
    max_prob_last_row = last_row.max()
    max_prob = max(max_prob_last_col, max_prob_last_row)
    if max_prob > highest_prob:
      highest_prob = max_prob
      most_likely_disease = disease
  return most_likely_disease, highest_prob
if (passed):
  disease, probability  = findMostLikely()
  print("The disease you most likley have is:",  disease, "with probability of: ", probability)



#for disease in diseases: # Loop through all diseases
#  result = inference.query(variables= symptom_list, evidence={disease_name: 1})
# Query the probability of a symptom given a disease

# phi value is probability:
# 0 means not having it, 1 means having it
# probabiltiy of 1 and 1
# find disease that matches symptoms
#result = inference.query(variables= symptom_list, evidence={disease_name: 1})
#print(result)

symptoms start with an S_ before the symptom. Ex: S_Body Weight
enter symptoms, separated by commas: S_Body Weight
The disease you most likley have is: D_Growth Disorders with probability of:  1.4599936348785912e-10
